In [27]:
from datetime import datetime
#import MetaTrader5 as mt5
import pandas as pd
import pytz
import numpy as np
from ta.volatility import BollingerBands
from ta.momentum import StochasticOscillator
from ta.trend import MACD
import plotly.express as px

In [28]:
# Importamos la informacion de MetaTrader5 y la almacenamos en un DataFrame
def import_data():
    if not mt5.initialize():
        print("initialize() failed, error code =",mt5.last_error())
    quit()
    # set time zone to UTC
    
    timezone = pytz.timezone("Etc/UTC")
    # create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
    utc_from = datetime(2020, 2, 1, tzinfo=timezone)
    utc_to = datetime(2021, 2, 1, tzinfo=timezone)
    # get bars from USDJPY M5 within the interval of 2020.01.10 00:00 - 2020.01.11 13:00 in UTC time zone
    rates = mt5.copy_rates_range("BITCOIN", mt5.TIMEFRAME_D1, utc_from, utc_to)

    
    # create DataFrame out of the obtained data
    rates_frame = pd.DataFrame(rates)
    # convert time in seconds into the 'datetime' format
    #rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
    
    rates_frame
    #rates_frame.to_csv('BTC20_21.csv')
    
    return rates_frame

In [29]:
#ENTRENAMIENTO
data_E = pd.read_csv('BTC19_20.csv')

In [30]:
#PRUEBA
data_P = pd.read_csv('BTC20_21.csv')

In [31]:
# Boolinger
# Empleamos la libreria de BollingerBands para obtener las senales de compra
def tec1(data):
    df = data.copy()
    indicator_bb = BollingerBands(close=data.close, window=21)
    df['bb_bbm'] = indicator_bb.bollinger_mavg()
    df['bb_bbh'] = indicator_bb.bollinger_hband()
    df['bb_bbl'] = indicator_bb.bollinger_lband()
    df['bb_high_signal'] = indicator_bb.bollinger_hband_indicator()
    df['bb_low_signal'] = indicator_bb.bollinger_lband_indicator()
    return df

# RSI
def tec2(data):
    df = data.copy()
    indicator_stoc = StochasticOscillator(high=data['high'], low=data['low'], close=data['close'])
    df['stochastic'] = indicator_stoc.stoch()
    df['stochastic_buy_signal'] = indicator_stoc.stoch() < 20
    df['stochastic_sell_signal'] = indicator_stoc.stoch() > 80
    return df

In [32]:
# Bandas de boolinger 
data_ES = tec1(data_E).dropna()
data_ES

,Unnamed: 0,time,open,high,low,close,tick_volume,spread,real_volume,bb_bbm,bb_bbh,bb_bbl,bb_high_signal,bb_low_signal
20,20,1548806400,3377.49,3435.44,3349.37,3408.75,4048838,0,6840090,3637.959048,4004.133975,3271.784120,0.0,0.0
21,21,1548892800,3408.75,3448.57,3372.38,3393.54,853227,0,6840127,3615.709524,3981.805279,3249.613769,0.0,0.0
22,22,1548979200,3393.54,3432.37,3350.26,3425.32,1652827,0,6835363,3599.250476,3967.022038,3231.478914,0.0,0.0
23,23,1549238400,3371.25,3411.60,3369.05,3393.46,2448519,0,6840262,3579.398571,3944.443706,3214.353436,0.0,0.0
24,24,1549324800,3393.46,3406.23,3375.81,3399.04,3250885,0,6840255,3551.663810,3876.601449,3226.726170,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,254,1577145600,7294.75,7412.25,7139.25,7180.25,340064,0,3765000,7235.271429,7688.730962,6781.811895,0.0,0.0
255,255,1577318400,7162.75,7419.25,7150.25,7173.75,278882,0,3240000,7239.290476,7688.985981,6789.594971,0.0,0.0
256,256,1577404800,7172.75,7240.75,7026.75,7204.50,322672,0,4308954,7224.019048,7655.242185,6792.795910,0.0,0.0
257,257,1577664000,7373.00,7412.00,7181.50,7231.75,279110,0,4308833,7209.305714,7616.728073,6801.883356,0.0,0.0


In [143]:
# Oscilador estocastico
data_ES2 = tec2(data_E).dropna()
data_ES2

,Unnamed: 0,time,open,high,low,close,tick_volume,spread,real_volume,stochastic,stochastic_buy_signal,stochastic_sell_signal
13,13,1548028800,3515.45,3544.94,3482.49,3497.50,2451643,0,6840151,4.344442,True,False
14,14,1548115200,3497.50,3587.53,3402.36,3549.88,10772862,0,6835359,21.535766,False,False
15,15,1548201600,3549.88,3589.22,3498.51,3521.29,1651960,0,6840203,17.362044,True,False
16,16,1548288000,3521.29,3572.07,3504.88,3547.24,4052829,0,6840137,21.150365,False,False
17,17,1548374400,3547.24,3554.75,3490.64,3534.75,2454167,0,6835422,19.327007,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
254,254,1577145600,7294.75,7412.25,7139.25,7180.25,340064,0,3765000,61.539946,False,False
255,255,1577318400,7162.75,7419.25,7150.25,7173.75,278882,0,3240000,61.038209,False,False
256,256,1577404800,7172.75,7240.75,7026.75,7204.50,322672,0,4308954,63.411810,False,False
257,257,1577664000,7373.00,7412.00,7181.50,7231.75,279110,0,4308833,65.515245,False,False


In [34]:
# retorno promedio
def Average(lst):
    return sum(lst) / len(lst)

In [145]:
def transactions(vol, pip_tp, pip_sl, df):
    rett=[]
    
    while len(df)>2:
        precio_transaccion=df['close'].iloc[0]
        posicion=precio_transaccion*vol
        day0=df.iloc[0,0]

        closed=df[(df['close']>=(precio_transaccion+pip_tp/100))|(df['close']<=(precio_transaccion-pip_sl/100))]
        closed_price=closed.iloc[0,5]
        closed_transaction= closed_price*vol
        day1=closed.iloc[0,0]

        ret=246*(closed_transaction/posicion-1)/(day1-day0)

        rett.append(ret)

        df=df.loc[closed.iloc[0,0]:]
    
    return rett

In [146]:
vol=0.5 # volumen en bitcoin
pips=9000 # exposicion por operacion 100 usd
df = data_ES2[(data_ES2['stochastic_buy_signal']==True) | (data_ES2['stochastic_sell_signal']==True)] # df con las señales de compra y venta
transactions(vol, pips,pips, df)

[-1.4703027877055015,
 2.10397821384152,
 0.39400277409900586,
 0.6795801800433627,
 24.74927206585117,
 -0.5697166764050448,
 2.946211955352939,
 4.203954588457898,
 0.6096923118458611,
 3.3736425872424114,
 4.989495870494304,
 8.504071697533842,
 61.80028338556395,
 -4.493974367149136,
 14.320135095638609,
 -1.618684618198521,
 -2.7058068491040688,
 4.3485230284638865,
 19.436288107484362,
 -1.4344323360321325,
 -4.555576687351342,
 7.270972323175853,
 -3.975517978309057,
 5.92351810771344,
 9.714489374905625,
 22.721584445968286,
 11.663534681397902,
 29.797978302313354,
 -4.349247116631223,
 2.4672604278214307,
 0.7710811719376253,
 -4.490008193251067,
 11.478195830988767,
 1.3770229177863043,
 -3.037253106030908,
 -3.6091799307492023,
 2.520134754407885,
 13.948782538308407,
 -1.5562600625154133,
 -29.976883442989326,
 -6.969144440352802,
 2.2083778431552386,
 -0.9125131174323661,
 -0.5527551626248527,
 -4.708483125643773,
 17.791965920245758,
 26.023448285544504,
 -1.650108981365

In [147]:
Average(transactions(vol, pips,pips, df))

3.9716361910195137

In [62]:
def capital(vol, pip_tp, pip_sl, df):
    dayss=[]
    cashh=[]
    cash=100000
    while len(df)>2:
        precio_transaccion=df['close'].iloc[0]
        posicion=precio_transaccion*vol
        cash=cash-posicion
        day0=df.iloc[0,0]

        closed=df[(df['close']>=(precio_transaccion+pip_tp/100))|(df['close']<=(precio_transaccion-pip_sl/100))]
        closed_price=closed.iloc[0,5]
        closed_transaction= closed_price*vol

        cash=cash+closed_transaction
        cashh.append(cash)
        df=df.loc[closed.iloc[0,0]:]
        dayss.append(day0)
    
    dff = pd.DataFrame(dict(
    days = dayss,
    capital = cashh))
    
    return dff

In [63]:
vol=4 # volumen en bitcoin
pip_tp=9000 # exposicion por operacion 100 usd
pip_sl=9000 # exposicion por operacion 100 usd
df = data_ES2[(data_ES2['stochastic_buy_signal']==True) | (data_ES2['stochastic_sell_signal']==True)] # df con las señales de compra y venta
a=capital(vol, pip_tp, pip_sl, df)
a

,days,capital
0,13,99581.92
1,18,101323.08
2,33,101813.60
3,53,102206.52
4,62,107094.96
5,65,106655.48
6,74,107150.00
7,76,108233.80
8,79,108674.44
9,87,109296.08


In [64]:
fig = px.line(a, x='days', y="capital", title="Capital") 
fig.show()

In [65]:
a.iloc[-1,1]

113428.00000000003

In [42]:
def capitalgrid(vol, pips, df):
    dayss=[]
    cashh=[]
    cash=100000
    while len(df)>2:
        precio_transaccion=df['close'].iloc[0]
        posicion=precio_transaccion*vol
        cash=cash-posicion
        day0=df.iloc[0,0]

        closed=df[(df['close']>=(precio_transaccion+pips/100))|(df['close']<=(precio_transaccion-pips/100))]
        closed_price=closed.iloc[0,5]
        closed_transaction= closed_price*vol

        cash=cash+closed_transaction
        cashh.append(cash)
        df=df.loc[closed.iloc[0,0]:]
        dayss.append(day0)
    
    dff = pd.DataFrame(dict(
    days = dayss,
    capital = cashh))
    
    return dff.iloc[-1,1]

In [43]:
vol=4 # volumen en bitcoin
pips=9000 # exposicion por operacion 100 usd
capitalgrid(vol, pips, df)

113428.00000000003

In [113]:
vol=np.arange(0.5,8,0.1)
pips=np.arange(1000,100000,1000)
pips = pips[0:96]

In [114]:
len(pips),len(vol),len(pips)*len(vol)

(99, 75, 7425)

In [121]:
# For i,j loop es un grid search
# Podemos usar un gridsearch sin PSO
def grid_search(vol, pip_sl, pip_tp, df):
    #Create dataframe
    dif_port = pd.DataFrame()
    sl_list = []
    tp_list = []
    max_loss_list = []
    Sharpe_list = []
    #Iterate with different parameteres
    for i in vol:
        for j in pip_sl:
            for k in pip_tp:
                ent_capital = capital(i, j, k, df)
                #Obtaining the sharpe ratio
                Portfolio = pd.DataFrame()
                Portfolio["Portfolio Value"] = ent_capital.iloc[:,1]
                Portfolio["Return"] = Portfolio["Portfolio Value"].pct_change(1)
                Sharpe= Portfolio["Return"].mean() /Portfolio["Return"].std()
                sl_list.append(j)
                tp_list.append(k)
                max_loss_list.append(i)
                Sharpe_list.append(Sharpe)  
    
    dif_port["Volume"] = max_loss_list
    dif_port["Stop Loss"] = sl_list
    dif_port["Take profit"] = tp_list
    dif_port["Sharpe"] = Sharpe_list
    return dif_port



In [130]:
# parametros postizos
vol=np.arange(1,8,1)
pips=np.arange(1000,100000,10000)
len(pips)*len(vol)

70

In [132]:
df = data_ES2[(data_ES2['stochastic_buy_signal']==True) | (data_ES2['stochastic_sell_signal']==True)] # df con las señales de compra y 
busqueda = grid_search(vol, pips, pips, df)
busqueda

In [134]:
busqueda

,Volume,Stop Loss,Take profit,Sharpe
0,1,1000,1000,0.062741
1,1,1000,11000,0.073480
2,1,1000,21000,0.080271
3,1,1000,31000,0.080774
4,1,1000,41000,0.086694
...,...,...,...,...
695,7,91000,51000,0.113847
696,7,91000,61000,0.112694
697,7,91000,71000,0.117030
698,7,91000,81000,0.104382


In [165]:
#buscar el de mayor sharpe y usar esos parametros en los datos de prueba
maxSharp = busqueda.Sharpe.max()
best_param = busqueda.loc[busqueda.Sharpe==maxSharp]

In [166]:
data_PS2 = tec2(data_P).dropna()
df_p = data_PS2[(data_PS2['stochastic_buy_signal']==True) | (data_PS2['stochastic_sell_signal']==True)] # df con las señales de compra y venta
a2=capital(best_param.iloc[0,0], best_param.iloc[0,1],best_param.iloc[0,2], df_p)
a2

,days,capital
0,16,96221.75
1,17,89776.50
2,25,75552.50
3,28,85345.08
4,45,97525.50
5,62,104950.75
6,69,102540.86
7,73,98600.00
8,109,111185.58
9,125,117955.00


In [167]:
transactions(best_param.iloc[0,0], best_param.iloc[0,1],best_param.iloc[0,2], df_p)

[-14.312269260826207,
 -3.2404077253218895,
 -21.316275945885444,
 3.4994571426392405,
 3.5051194810335478,
 4.177367778600443,
 -2.120547348390004,
 -0.39905998093406464,
 3.0453416508303723,
 2.7343977438555105,
 -5.826848782959332,
 -0.5620688909550885,
 -1.4876275595277784,
 -7.19582124201974,
 -3.499987671063977,
 1.1058434312785073,
 5.062749085270477,
 2.2081677112936537,
 21.722512337127394,
 2.957370165615278,
 7.4884669142532045,
 4.437919348834675,
 -2.2165586168620175,
 2.4873769415169678,
 -4.2316901779869545,
 2.5380048263710795,
 18.33097524097527,
 6.821133667471966,
 10.729431351502958,
 9.070825775368336,
 21.989408670125414,
 15.664716638148544,
 25.892723336975912,
 -1.200103788664491]

In [168]:
Average(transactions(best_param.iloc[0,0], best_param.iloc[0,1],best_param.iloc[0,2], df_p))

3.1723541837556404

In [171]:
fig2 = px.line(a2, x='days', y="capital", title="Evolución capital prueba") 
fig2.show()

In [47]:
# Primera transaccion
while len(df)>2:
    precio_transaccion=df['close'].iloc[0]
    posicion=precio_transaccion*vol
    day0=df.iloc[0,0]

    closed=df[(df['close']>=(precio_transaccion+pips/100))|(df['close']<=(precio_transaccion-pips/100))]
    closed_price=closed.iloc[0,5]
    closed_transaction= closed_price*vol
    day1=closed.iloc[0,0]

    ret=246*(closed_transaction/posicion-1)/(day1-day0)

    rett.append(ret)

    df=df.loc[closed.iloc[0,0]:]



ValueError: ('Lengths must match to compare', (96,), (99,))

In [26]:
Average(rett)

NameError: name 'rett' is not defined

In [ ]:
df_ret=pd.DataFrame(index = range(1,29), columns = ['Dias','Ret_anual', 'Std_anual','Radio_Sharpe'])
rf=0.045
for i in range(28):
    # Rendimientos si compraramos y vendieramos despues de i+1 dias
    ret_vec=data['close'].pct_change(i+1).dropna()
    ret_anual= ret_vec.mean()*(252/(i+1))
    std_anual=ret_vec.std()*np.sqrt((252/(i+1)))
    rs=(ret_anual-rf)/std_anual
    df_ret.iloc[i]=[i+1, ret_anual, std_anual, rs]
    
df_ret